# Image Generation Using gpt-image-1 model

## Setup 

```{python}
# Load the API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# Initialize the OpenAI client
client = OpenAI(api_key=api_key)
```

## Prompt

```{python}
# Ask the user to input a prompt in the terminal
print("What do you want to generate?")
prompt = input("> ")
print("Generating image...")
# Send the prompt to the API
img = client.images.generate(
  model="gpt-image-1",
  prompt=prompt,
  background="auto",
  n=1,
  quality="high",
  size="1024x1024",
  output_format="png",
  moderation="auto",
)
```

## Saving the Gnerated Image

```{python}
# Ask the user to input a prompt in the terminal
print("What do you want to generate?")
prompt = input("> ")
print("Generating image...")
# Send the prompt to the API
img = client.images.generate(
  model="gpt-image-1",
  prompt=prompt,
  background="auto",
  n=1,
  quality="high",
  size="1024x1024",
  output_format="png",
  moderation="auto",
)
```

## Running the Script

```bash
python generate_image.py
```
```bash
(gpt-images) francoisaubry@Francoiss-MBP gpt-image-1-demo % python generate_image.py
What do you want to generate?
> A busy city with animals dressed as people.
Generating image ...
o (gpt-images) francoisaubry@Francoiss-MBP gpt-image-1-demo %
```
![](../resources/img/gpt-image-1-1.png)


## GPT-Image 1-parameters

In this section, we describe the most relevant parameters of the gpt-image-1 model:

* **prompt**: The textual instruction that describes what image we want the model to generate.
* **background**: The type of background for the image. It must be one of "`transparent`", "`opaque`", or "`auto`". The default value is "`auto`", in which the model will decide based on the content what the best background type is. *Note that JPEG images don't support transparent backgrounds*.
* **n**: The number of images to generate. Must be a number from 1 to 10.
* **quality**: The quality of the generated image. It must be one of "`high`", "`medium`", or "`low`", with the default being "`high`".
* **size**: The size of the image in pixels. It must be one of "`1024x1024`" (square), "`1536x1024`" (landscape), "`1024x1536`" (portrait).
* **output_format**: The image format. It must be one of "`png`", "`jpeg`", or "`webp`". The default is "`png`".
* **moderation**: The level of content moderation. Must be either "`low`" for a less restrictive filter or "`auto`", which is the default.

We covered the most important parameters available for the gpt-image-1 model. For more details and a full list of parameters, check the official API documentation.

## Quality Parameter

![](../resources/img/gpt-image-1-quality.png)

We see that the first cat (with the lowest quality) has some artifacts around the eyes, and that the image becomes significantly more realistic as the quality increases.

![](../resources/img/gpt-image-2-quality.png)

In this case, we also notice that the higher the quality, the more polished the image is. In my opinion, however, this is less relevant for content that's not supposed to be realistic.

*Remember that the higher the quality, the higher the cost and the computation time, so it's a good idea to figure out a good trade-off, depending on the use case.* I'd recommend trying the parameters from low to high to see the minimum value that gives acceptable results for each use case.